In [23]:
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '..')

# for saving
import os

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from src import setup
# setup.use_gpu()

In [24]:
from src import ( 
    measurements as measure,
    density_matrix as DM,
    simulation as sim,
    orders,
    order_rules,
    random_unitary,
    simulation)

from Scripts import simulation_CLI as cleo

In [25]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.2, 0.23, 0.36, 0.15, 0.18, 0.408125]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.2, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 20
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generator_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random


In [26]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/random")

In [27]:
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors 

import matplotlib.animation as animation
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '../..')
import h5py
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from src import ket as ket
from src import density_matrix as DM

In [28]:
newdata1 = (h5py.File("../data/random/random.hdf5"))

In [29]:
def get_pops(data, n_qubits, connectivity,update_rule):
    #dimension 0 is each trial
    #dimension 1 is each time step
    #dimension 2 is each qubit
    result = []
    for trial in data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1']:
        seed = trial.split(' ')[-1]
        dat = dict(data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1'][f'unitary seed {seed}'][f'ordering seed {update_rule}']['pops'])
        dat = {int(k.split('(')[0]): dat[k] for k in dat}
        dat = np.array([np.array([dat[k][subkey][()] for subkey in sorted(dat[k])]) for k in sorted(dat)])
        result.append(dat)
    return(np.array(result))

def get_2_qbit_dms(data, n_qubits, connectivity,update_rule):
    basis = ket.canonical_basis(2)
    #dimension 0 is each trial
    #dimension 1 is each time step (recall that sampling step is 5 by default)
    #dimension 2 is qubit pair
    #dimension 3 and 4 is the 2 qubit density matrix
    result = []
    
    def to_tuple(string):
        tuple_elements = string.strip('()').split(',')
        return tuple(int(elem.strip()) for elem in tuple_elements)

    for trial in data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1']:
        seed = trial.split(' ')[-1]
        dat = dict(data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1'][f'unitary seed {seed}'][f'ordering seed {update_rule}']['two_qubit_dms'])
        dat = {int(k.split('(')[0]): dat[k] for k in dat}
        
        dat = np.array([{to_tuple(subkey):DM.DensityMatrix(dat[k][subkey],basis) for subkey in sorted(dat[k])} for k in sorted(dat)])
        result.append(dat)
    return(np.array(result))

In [9]:
def print_hdf5_structure(file_path):
    with h5py.File(file_path, 'r') as file:
        def print_group(name, obj):
            if isinstance(obj, h5py.Group):
                print(f'Group: {name}')
                for key in obj.keys():
                    print(f'  {key}')
            elif isinstance(obj, h5py.Dataset):
                print(f'Dataset: {name}')

        file.visititems(print_group)

# Replace with the path to your merged HDF5 file
file_path = '../data/random/random.hdf5'  
print_hdf5_structure(file_path)

Group: 8 qubits
  c2_2local connectivity
Group: 8 qubits/c2_2local connectivity
  unitary energy subspace 1
Group: 8 qubits/c2_2local connectivity/unitary energy subspace 1
  unitary seed 0
Group: 8 qubits/c2_2local connectivity/unitary energy subspace 1/unitary seed 0
  ordering seed random
Group: 8 qubits/c2_2local connectivity/unitary energy subspace 1/unitary seed 0/ordering seed random
  pops
  previous_order
  two_qubit_dms
Group: 8 qubits/c2_2local connectivity/unitary energy subspace 1/unitary seed 0/ordering seed random/pops
  0
  1
Group: 8 qubits/c2_2local connectivity/unitary energy subspace 1/unitary seed 0/ordering seed random/pops/0
  0
  1
  2
  3
  4
  5
  6
  7
Dataset: 8 qubits/c2_2local connectivity/unitary energy subspace 1/unitary seed 0/ordering seed random/pops/0/0
Dataset: 8 qubits/c2_2local connectivity/unitary energy subspace 1/unitary seed 0/ordering seed random/pops/0/1
Dataset: 8 qubits/c2_2local connectivity/unitary energy subspace 1/unitary seed 0/orderi

In [38]:
import h5py

def load_orders_list(file_path: str, num_qubits, connectivity, unitary_energy_subspace, unitary_seed, order_rule_name: str):
    # Create an empty dictionary to hold orders_list
    orders_list = {}

    with h5py.File(file_path, 'r') as file:
        # Construct the path to the specific group where orders_list is stored
        group_name = f"{num_qubits} qubits/{connectivity} connectivity/unitary energy subspace {unitary_energy_subspace}/unitary seed {unitary_seed}/ordering seed {order_rule_name}/previous_order"
        
        # Check if the group exists
        if group_name in file:
            group = file[group_name]
            for time_index in group:
                # Assuming the order data is stored in a dataset named 'data'
                orders_list[time_index] = group[time_index]['data'][()]
        else:
            print(f"Group '{group_name}' not found in file.")
    
    return orders_list



In [39]:
# Set your parameters
file_path = "../data/random/random.hdf5"  # Path to your combined HDF5 file
num_qubits = 8
connectivity = 'c2_2local'  # Replace with actual connectivity value
unitary_energy_subspace = '1'  # Replace with actual value
unitary_seed = '0'  # Replace with actual value
order_rule_name = 'random'  # Replace with actual rule

# Load orders_list
orders_list = load_orders_list(file_path, num_qubits, connectivity, unitary_energy_subspace, unitary_seed, order_rule_name)

# Print the loaded orders_list


In [44]:
output=orders_list
o

{'orders_list': array([[[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 7],
         [1, 2],
         [3, 4],
         [5, 6]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 7],
         [1, 2],
         [3, 4],
         [5, 6]],
 
        [[0, 7],
         [1, 2],
         [3, 4],
         [5, 6]],
 
        [[0, 7],
         [1, 2],
         [3, 4],
         [5, 6]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2, 3]],
 
        [[0, 1],
         [6, 7],
         [4, 5],
         [2

In [ ]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.2, 0.23, 0.36, 0.15, 0.18, 0.408125]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.2, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

[[0 1]
 [4 6]
 [2 3]
 [5 7]]
[[2 3]
 [1 7]
 [4 5]
 [0 6]]
[[0 1]
 [4 6]
 [2 3]
 [5 7]]
[[0 1]
 [4 6]
 [2 3]
 [5 7]]
[[0 2]
 [4 6]
 [1 3]
 [5 7]]
[[0 1]
 [6 7]
 [4 5]
 [2 3]]
[[1 7]
 [2 4]
 [3 5]
 [0 6]]
[[4 6]
 [0 7]
 [1 2]
 [3 5]]
[[0 7]
 [1 2]
 [3 4]
 [5 6]]
[[0 1]
 [6 7]
 [2 4]
 [3 5]]
[[0 1]
 [6 7]
 [2 4]
 [3 5]]
[[0 1]
 [6 7]
 [4 5]
 [2 3]]
[[4 5]
 [6 7]
 [0 2]
 [1 3]]
[[0 2]
 [5 6]
 [3 4]
 [1 7]]
[[0 2]
 [4 6]
 [3 5]
 [1 7]]
[[4 6]
 [0 7]
 [1 2]
 [3 5]]
[[0 2]
 [4 6]
 [1 3]
 [5 7]]
[[0 2]
 [4 6]
 [3 5]
 [1 7]]
[[2 3]
 [1 7]
 [4 5]
 [0 6]]
[[0 7]
 [1 2]
 [3 4]
 [5 6]]
[[2 3]
 [1 7]
 [4 5]
 [0 6]]
[[0 7]
 [2 4]
 [1 3]
 [5 6]]
[[0 7]
 [1 2]
 [3 4]
 [5 6]]
[[0 2]
 [4 6]
 [1 3]
 [5 7]]
[[2 3]
 [1 7]
 [4 5]
 [0 6]]
[[2 4]
 [1 3]
 [5 7]
 [0 6]]
[[1 7]
 [2 4]
 [3 5]
 [0 6]]
[[1 2]
 [3 4]
 [5 7]
 [0 6]]
[[0 1]
 [4 6]
 [2 3]
 [5 7]]
[[0 2]
 [5 6]
 [3 4]
 [1 7]]
[[0 1]
 [6 7]
 [2 4]
 [3 5]]
[[0 1]
 [6 7]
 [2 4]
 [3 5]]
[[0 7]
 [1 2]
 [3 4]
 [5 6]]
[[0 7]
 [1 2]
 [3 4]
 [5 6]]
[[1 7]
 [2 4]


In [ ]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.2, 0.23, 0.36, 0.15, 0.18, 0.408125]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
   # initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.2, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

In [ ]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.2, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

In [ ]:
Trials = 100
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.2, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

In [8]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/random")

In [8]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [9]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [10]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [11]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [12]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [14]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/greedy")

In [13]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:56: RuntimeWarning: invalid value encountered in log
  return 1 / (np.log((1 - pop) / pop))
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: divide by zero encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in scalar multiply
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)


90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [14]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [15]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [16]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [17]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [20]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/mimic")

In [18]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [19]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [20]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [21]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)


90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [22]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
